Prepare Sequence Data

In [1]:
import pandas as pd
import numpy as np
import pygtop
import requests
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
import json

In [2]:
endo_data = pd.read_csv("input_files/peptide_ligands_Arman.csv")
print(endo_data)

     Receptor Name Receptor Accession    Ligand Name  \
0      5ht1b_human             P28222  5-HT-moduline   
1      5ht1b_human             P28222  5-HT-moduline   
2      5ht1b_human             P28222  5-HT-moduline   
3      5ht1d_human             P28221  5-HT-moduline   
4      5ht1d_human             P28221  5-HT-moduline   
...            ...                ...            ...   
2877      oxyr_rat             P70536       oxytocin   
2878      oxyr_rat             P70536       oxytocin   
2879      oxyr_rat             P70536       oxytocin   
2880      oxyr_rat             P70536       oxytocin   
2881      oxyr_rat             P70536       oxytocin   

                         InChIKey  Smiles   Sequence  Alternate ID  \
0     IMIVWAUMTAIVPJ-XUXIUFHCSA-N     NaN       LSAL       7048565   
1     IMIVWAUMTAIVPJ-XUXIUFHCSA-N     NaN       LSAL           137   
2     IMIVWAUMTAIVPJ-XUXIUFHCSA-N     NaN       LSAL       7048566   
3     IMIVWAUMTAIVPJ-XUXIUFHCSA-N     NaN      

In [3]:
spec_lig_data = pd.read_csv("input_files/peptide_specligands_Arman.csv", index_col=0)
print(spec_lig_data)

             gtp_id                        chainA_name  \
ligand_name                                              
INSL3          1995              Insuline-like A chain   
LH             1159              Lutropin subunit beta   
relaxin-3      1990                            A chain   
relaxin-1      1988                            A chain   
TSH            3920           Thyrotropin subunit beta   
INSL5          2000       Insulin-like peptide B chain   
hCG            1160  Choriogonadotropin subunit beta 3   
thrombin       4453                        light chain   
relaxin        1989                            A chain   

                                                    chainA_seq chainA_id  \
ligand_name                                                                
INSL3                               AAATNPARYCCLSGCTQQDLLTLCPY    P51460   
LH           SREPLRPWCHPINAILAVEKEGCPVCITVNTTICAGYCPTMMRVLQ...    P01229   
relaxin-3                             DVLAGLSSSCCKWGCSKSE

In [4]:
# manually adding GPR39_human/neurotensin complex and GRP15/GRP15L

fields = {'Receptor Name': "rec", 'Receptor Accession': "rec", 'Ligand Name': "lig",
          'InChIKey': "lig", 'Smiles': "lig", 'Sequence': "lig", 'Alternate ID': "lig",
          'Source': "lig", #'Receptor Sequence': "rec"
          }

for rec_name, rec_acc, lig_name, lig_gtp in [("gpr39_human", "O43194", "neurotensin", "1579"), # neurotensin
                          ("gpr15_human", "P49685", "GPR15L", "10567"), # GRP15L full length
                          ("gpr15_human", "P49685", "GPR15L_71-81", "10568") # GRP15L shorter version
                          ]:
    try:
        rec_row = endo_data[endo_data["Receptor Name"] == rec_name].iloc[0, :]
    except IndexError:
        rec_row = pd.Series(index=endo_data.columns)
        rec_row["Receptor Name"] = rec_name
        rec_row["Receptor Accession"] = rec_acc
    try:
        lig_row = endo_data[endo_data["Alternate ID"] == lig_gtp].iloc[0, :]
    except IndexError:
        lig_row = pd.Series(index=endo_data.columns)
        lig_row["Ligand Name"] = lig_name
        lig_row["Alternate ID"] = lig_gtp
        lig_row["Source"] = "Guide To Pharmacology"
    new_row = rec_row.copy()
    for key, from_row in fields.items():
        if from_row == "rec":
            new_row[key] = rec_row[key]
        else:
            new_row[key] = lig_row[key]
    print()
    print(new_row)
    endo_data.loc[len(endo_data.index)] = new_row


Receptor Name                         gpr39_human
Receptor Accession                         O43194
Ligand Name                           neurotensin
InChIKey              PCJGZPGTCUMMOT-ISULXFBGSA-N
Smiles                                        NaN
Sequence                            XLYENKPRRPYIL
Alternate ID                                 1579
Source                      Guide To Pharmacology
Name: 644, dtype: object

Receptor Name                   gpr15_human
Receptor Accession                   P49685
Ligand Name                          GPR15L
InChIKey                                NaN
Smiles                                  NaN
Sequence                                NaN
Alternate ID                          10567
Source                Guide To Pharmacology
dtype: object

Receptor Name                   gpr15_human
Receptor Accession                   P49685
Ligand Name                    GPR15L_71-81
InChIKey                                NaN
Smiles                        

/var/folders/ls/ljr1g7ps2xd3yb1pnstmg4440000gn/T/ipykernel_13447/1094284396.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  rec_row = pd.Series(index=endo_data.columns)
/var/folders/ls/ljr1g7ps2xd3yb1pnstmg4440000gn/T/ipykernel_13447/1094284396.py:21: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lig_row = pd.Series(index=endo_data.columns)
/var/folders/ls/ljr1g7ps2xd3yb1pnstmg4440000gn/T/ipykernel_13447/1094284396.py:21: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lig_row = pd.Series(index=endo_data.columns)


In [5]:
column_name = "Sequence"
counter = 0
ligand_list = list(set(endo_data["Ligand Name"]))

# loop over all ligands to check for sequence
for lig in ligand_list:
    lig_data = endo_data[endo_data["Ligand Name"] == lig]

    # ligand has sequence but not for all entries
    if pd.isna(lig_data["Sequence"]).any() and not pd.isna(lig_data["Sequence"]).all():
        print("> Added partially missing sequences to ligand '%s' entries " % lig)
        # get sequence from first ligand entry with a sequence
        sequence = lig_data[~pd.isna(lig_data["Sequence"])]["Sequence"].iloc[0]
        # set sequence for all ligand entries without sequence
        endo_data.loc[endo_data["Ligand Name"] == lig, "Sequence"] = sequence
    if pd.isna(lig_data["Sequence"]).all():
        if lig in spec_lig_data.index:
            print("> Ligand '%s' is in special set (has multiple chains). Getting sequences from there" % lig)
            lig_entry = str(spec_lig_data.loc[lig, :].to_dict())
            endo_data.loc[endo_data["Ligand Name"] == lig, "Sequence"] = lig_entry
            continue
        print(">  Sequences for ligand '%s' missing " % lig)
        counter += 1
        gtp_id = int(lig_data[lig_data["Source"] == "Guide To Pharmacology"]["Alternate ID"].values[0])

        print("   > Guide to Pharmacology ID is %s" % gtp_id)
        my_drug = pygtop.get_ligand_by_id(gtp_id)

        sequence = my_drug.one_letter_sequence()
        endo_data.loc[endo_data["Ligand Name"] == lig, "Sequence"] = sequence

        db_links = my_drug.database_links()
        for db_link in db_links:
            db_link = db_link.json_data
            if db_link["database"] == "UniProtKB":
                uniprot_id = db_link["accession"]
                print("   > UniProt accession is %s" % uniprot_id)
            elif db_link["database"] == "PubChem":
                pubchem_id = db_link["accession"]
                print("   > PubChem accession is %s" % pubchem_id)
            else:
                continue

print("\n> %i ligands have missing sequences" % counter)

>  Sequences for ligand 'GPR15L_71-81' missing 
   > Guide to Pharmacology ID is 10568
> Ligand 'relaxin' is in special set (has multiple chains). Getting sequences from there
> Added partially missing sequences to ligand 'FSH' entries 
> Ligand 'INSL3' is in special set (has multiple chains). Getting sequences from there
> Ligand 'thrombin' is in special set (has multiple chains). Getting sequences from there
> Ligand 'relaxin-1' is in special set (has multiple chains). Getting sequences from there
> Ligand 'LH' is in special set (has multiple chains). Getting sequences from there
> Ligand 'INSL5' is in special set (has multiple chains). Getting sequences from there
> Ligand 'TSH' is in special set (has multiple chains). Getting sequences from there
> Ligand 'hCG' is in special set (has multiple chains). Getting sequences from there
>  Sequences for ligand 'GPR15L' missing 
   > Guide to Pharmacology ID is 10567
   > UniProt accession is Q6UWK7
> Ligand 'relaxin-3' is in special set (

In [6]:
receptor_list = list(set(endo_data["Receptor Name"].values))

def getSequence(protein_id):
    #print(" > Querying GPCRdb for protein '%s' " % protein_id)
    url = "https://gpcrdb.org/services/protein/"+protein_id
    #print("   "+url)
    response = requests.get(url)
    protein_data = response.json()
    return protein_data['sequence']

# Getting receptor sequences from GPCRdb
for rec in receptor_list:
    try:
        sequence = getSequence(rec)
    except KeyError:
        print("> Sequence of '%s' could not be retrieved." % rec)
        pass
    endo_data.loc[endo_data["Receptor Name"] == rec, "Receptor Sequence"] = sequence

> Sequence of 'ednrb_rat' could not be retrieved.


In [7]:
# manually adding receptor sequence for one missing entry on GPCRdb
for entry, fastafile in [("ntr2_mouse", "input_files/P70310_NTR2_mouse.fasta")]:
    record = SeqIO.parse(open(fastafile, "r"), "fasta")
    sequence = str(list(record)[0].seq)
    endo_data.loc[endo_data["Receptor Name"] == entry, "Receptor Sequence"] = sequence

In [8]:
print("> %i receptors have missing sequences" % list(pd.isna(endo_data["Receptor Sequence"])).count(True))

> 0 receptors have missing sequences


In [9]:
endo_data_unique = endo_data[endo_data["Source"] == "Guide To Pharmacology"]
endo_data_unique = endo_data_unique.drop_duplicates(subset=["Receptor Name", "Sequence"])
print(" > %i unique complex pairs to model" % len(endo_data_unique))
endo_data_unique.to_csv("input_files/peptide_ligands_Arman_completed.csv")

 > 736 unique complex pairs to model


Prepare AlphaFold Jobs

In [10]:
# helper function to (quick and dirty) generate AF scripts for ILF cluster
def af_ilf_jobscript(jobname,
                     fastafile,
                     jobscript_file,
                     only_compute_MSAs,
                     use_precompute_MSAs,
                     working_dir="$PWD",
                     singleton_dependency=False,
                     do_amber=True,
                     ncores=6,
                     mem=20):
    af_dir = "/projects/ilfgrid/apps/alphafold-2.3.1"
    af_db_dir = "/projects/ilfgrid/data/alphafold-genetic-databases/"
    if only_compute_MSAs:
        cp_cmd = ""
        gswitch = "#SBATCH --exclude=ilfgridgpun02fl"
        af_cmd = f"cd $AF_DIR && /projects/ilfgrid/apps/alphafold-2.2.0/prep_MSAs.sh -d $AFDB_DIR -n {ncores} -m {mem} -o $WDIR/ -f $FASTAFILE"

    else:
        cp_cmd = "cp $WDIR/$BASENAME/ranked_0.pdb $WDIR/${BASENAME}.pdb"
        gswitch = "#SBATCH --gres=gpu:1                                                         # no. of gpus (can only use one)"
        gpu_check = ""
        af_cmd = f"cd $AF_DIR && $AF_DIR/run_alphafold.sh -d $AFDB_DIR -n {ncores} -o $WDIR/ -f $FASTAFILE -t 2023-01-01 -m multimer -r {str(do_amber).lower()} -l 1 -e false -p {str(use_precompute_MSAs).lower()} -a $SLURM_JOB_GPUS"


    if singleton_dependency:
        dependency = "#SBATCH --dependency=singleton"
    else:
        dependency = ""

    command = f'''#!/bin/bash -l
#SBATCH --job-name={jobname}
###SBATCH --output={working_dir}/slurm-%A.out
###SBATCH --chdir={af_dir}/                    # change location to the install directory
#SBATCH --partition=standard                                                 # the queue you submit to
#SBATCH --mem={mem}G                                                            # the amount of memory to reserve
#SBATCH --ntasks={ncores}                                                          # combined with --cpus-per-task it determines
#SBATCH --cpus-per-task=1                                                    # the number of cpus to use (here: 12)
#SBATCH --nodes=1                                                            # no. of nodes to use
{gswitch}
{dependency}
echo ">> Job started at: $(date)"
echo "   On Machine:     $(hostname)"
echo -e "   In directory:   $(pwd)\\n"
echo -e ">> GPU is \"${{SLURM_JOB_GPUS}}\"\\n"
start=`date +%s`

WDIR="{working_dir}"
FASTAFILE="$WDIR/{fastafile}"

AF_DIR="{af_dir}"
AFDB_DIR="{af_db_dir}/"
module load miniconda
conda activate $AF_DIR/AF_2.3.1_conda_env

'''

    command = command+af_cmd

    command = command+f'''

BASENAME=`basename $FASTAFILE .fasta`
{cp_cmd}

end=`date +%s`
runtime=$((end-start))
echo -e \"\\n>> Job finished at: $(date)\"
echo -e \"\\n>> Runtime: $runtime s\"'''

    with open(os.path.join(jobscript_file), "w+") as f:
        f.write(command)

In [11]:
endo_data_unique = pd.read_csv("input_files/peptide_ligands_Arman_completed.csv", index_col=0)
print(endo_data_unique)

     Receptor Name Receptor Accession    Ligand Name  \
1      5ht1b_human             P28222  5-HT-moduline   
4      5ht1d_human             P28221  5-HT-moduline   
6       c3ar_human             Q16581            C3a   
7       c3ar_human             Q16581            C3a   
8       c3ar_human             Q16581            C5a   
...            ...                ...            ...   
2846    oxyr_human             P30559       oxytocin   
2864      oxyr_rat             P70536       oxytocin   
2882   gpr39_human             O43194    neurotensin   
2883   gpr15_human             P49685         GPR15L   
2884   gpr15_human             P49685   GPR15L_71-81   

                         InChIKey  Smiles  \
1     IMIVWAUMTAIVPJ-XUXIUFHCSA-N     NaN   
4     IMIVWAUMTAIVPJ-XUXIUFHCSA-N     NaN   
6                             NaN     NaN   
7                             NaN     NaN   
8                             NaN     NaN   
...                           ...     ...   
2846  XNOPRX

In [12]:
endo_data_unique["has_X"] = False
for index, row in endo_data_unique.iterrows():
    if "X" in row["Receptor Sequence"] or "X" in row["Sequence"]:
        endo_data_unique.loc[index, "has_X"] = True

print("Entries with X in sequences:", list(endo_data_unique["has_X"].values).count(True))

Entries with X in sequences: 42


In [13]:
# build directories and make AF jobs
af_dir = "af_jobs_af23"
os.makedirs(af_dir, exist_ok=True)
for i, row in endo_data_unique.iterrows():
    # Make directories
    rec = row["Receptor Name"]
    rec_dir = os.path.join(af_dir, rec)
    os.makedirs(rec_dir, exist_ok=True)

    lig = str(row["Alternate ID"])
    complex_dir = os.path.join(rec_dir, lig)
    complex = "%s-%s" % (rec, lig)
    os.makedirs(os.path.join(complex_dir), exist_ok=True)
    if os.path.exists(os.path.join(complex_dir, "%s.pdb" % complex)):
        continue

    # save info about complex to json
    row.to_json(os.path.join(complex_dir, "%s.json" % complex))

    # prepare sequence file for AF
    rec_record = SeqRecord(Seq(row["Receptor Sequence"]),
                           id=row["Receptor Name"],
                           description=row["Receptor Accession"])

    # check if multi-chain ligand
    try:
        multi_lig_entry = eval(row["Sequence"])
    except NameError:
        # not a multi-chain ligand
        lig_record = SeqRecord(Seq(row["Sequence"]),
                           id=row["Ligand Name"],
                           description="gtp:%i" % row["Alternate ID"])
        records = [rec_record, lig_record]
    else:
        # multi-chain ligand
        print(" >>> Complex %s has a multi-chain ligand" % complex)
        print(multi_lig_entry)
        json_file = os.path.join(complex_dir, "multi_chain_ligand.json")
        json.dump(multi_lig_entry, open(json_file, "w+"), indent=4)

        lig1_record = SeqRecord(Seq(str(multi_lig_entry["chainA_seq"])),
                   id=multi_lig_entry["chainA_name"],
                   description="| uniprot:%s | part of ligand '%s' with gtp:%s" % (multi_lig_entry["chainA_id"], row["Ligand Name"], multi_lig_entry["gtp_id"]))
        lig2_record = SeqRecord(Seq(str(multi_lig_entry["chainB_seq"])),
                   id=multi_lig_entry["chainB_name"],
                   description="| uniprot:%s | part of ligand '%s' with gtp:%s" % (multi_lig_entry["chainB_id"], row["Ligand Name"], multi_lig_entry["gtp_id"]))
        records = [rec_record, lig1_record, lig2_record]

    # save sequence file
    fastafile = os.path.join(complex_dir, "%s.fasta" % complex)
    SeqIO.write(records, open(fastafile, "w+"), "fasta")

    # make AF scripts
    # first script to blast DBs and make MSAs (CPU only)
    af_ilf_jobscript(jobname=complex,
                     fastafile=os.path.basename(fastafile),
                     jobscript_file=os.path.join(complex_dir, "0_prepMSAs.run"),
                     use_precompute_MSAs=False, only_compute_MSAs=True,
                     ncores=2)

    do_amber = False if row["has_X"] else True

    # second script to build AF models (needs GPU)
    af_ilf_jobscript(jobname=complex,
                     fastafile=os.path.basename(fastafile),
                     jobscript_file=os.path.join(complex_dir, "1_buildmodels.run"),
                     use_precompute_MSAs=True, only_compute_MSAs=False,
                     singleton_dependency=True,
                     do_amber=do_amber,
                     ncores=4)

    print(" > Scripts for complex %s generated " % complex)

 >>> Complex rxfp1_human-1988 has a multi-chain ligand
{'gtp_id': 1988, 'chainA_name': 'A chain', 'chainA_seq': 'PYVALFEKCCLIGCTKRSLAKYC', 'chainA_id': 'P04808', 'chainB_name': 'B chain', 'chainB_seq': 'VAAKWKDDVIKLCGRELVRAQIAICGMSTWS', 'chainB_id': 'P04808'}
 > Scripts for complex rxfp1_human-1988 generated 
 >>> Complex rxfp1_human-1989 has a multi-chain ligand
{'gtp_id': 1989, 'chainA_name': 'A chain', 'chainA_seq': 'QLYSALANKCCHVGCTKRSLARFC', 'chainA_id': 'P04090', 'chainB_name': 'B chain', 'chainB_seq': 'DSWMEEVIKLCGRELVRAQIAICGMSTWS', 'chainB_id': 'P04090'}
 > Scripts for complex rxfp1_human-1989 generated 
 >>> Complex rxfp1_human-1990 has a multi-chain ligand
{'gtp_id': 1990, 'chainA_name': 'A chain', 'chainA_seq': 'DVLAGLSSSCCKWGCSKSEISSLC', 'chainA_id': 'Q8WXF3', 'chainB_name': 'B chain', 'chainB_seq': 'RAAPYGVRLCGREFIRAVIFTCGGSRW', 'chainB_id': 'Q8WXF3'}
 > Scripts for complex rxfp1_human-1990 generated 
 >>> Complex rl3r1_human-2000 has a multi-chain ligand
{'gtp_id': 2000,

# If generated locally, move whole filesystem generated under ./af_jobs to the cluster. Then use go into ./af_jobs on the cluster and submit the jobs with "WD=$PWD; for JOBDIR in */*; do cd $JOBDIR; cd $WD; sbatch < 0_prepMSAs.run; sbatch < 1_buildmodels.run; done" or similar